In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import pprint
# from statesjson import us_states_json
# from states import us_states
from citipy import citipy

In [8]:
pollution_df = pd.read_csv("Air Quality Cleaner.csv").rename(columns= {"latitude": "Latitude", "longitude": "Longitude"})
PC = pd.read_csv("Park Coordinates.csv")
pcrounded = PC.round({"Latitude": 4, "Longitude": 3})

In [9]:
def cartesian_product_basic(left, right):
    return (
       pollution_df.assign(key=1).merge(pcrounded.assign(key=1), on='key').drop('key', 1))
pollution_merged_df = cartesian_product_basic(pollution_df, pcrounded)
#Credit: Umair Khakoo

In [10]:
pollution_merged_df

,Latitude_x,Longitude_x,date_local,aqi,local_site_name,site_address,state,county,city,Latitude_y,Longitude_y,Park
0,41.975804,-70.023598,2019-01-15,34.0,TRURO NATIONAL SEASHORE,FOX BOTTOM AREA,Massachusetts,Barnstable,Truro,44.3386,-68.273,Acadia National Park
1,41.975804,-70.023598,2019-01-15,34.0,TRURO NATIONAL SEASHORE,FOX BOTTOM AREA,Massachusetts,Barnstable,Truro,-14.2583,-170.683,National Park of American Samoa
2,41.975804,-70.023598,2019-01-15,34.0,TRURO NATIONAL SEASHORE,FOX BOTTOM AREA,Massachusetts,Barnstable,Truro,38.7331,-109.593,Arches National Park
3,41.975804,-70.023598,2019-01-15,34.0,TRURO NATIONAL SEASHORE,FOX BOTTOM AREA,Massachusetts,Barnstable,Truro,43.8554,-102.340,Badlands National Park
4,41.975804,-70.023598,2019-01-15,34.0,TRURO NATIONAL SEASHORE,FOX BOTTOM AREA,Massachusetts,Barnstable,Truro,29.1275,-103.243,Big Bend National Park
...,...,...,...,...,...,...,...,...,...,...,...,...
36327,47.048537,-109.455315,2019-01-15,28.0,Lewistown,303 East Aztec Drive Lewistown MT 59457,Montana,Fergus,Not in a city,43.6046,-103.421,Wind Cave National Park
36328,47.048537,-109.455315,2019-01-15,28.0,Lewistown,303 East Aztec Drive Lewistown MT 59457,Montana,Fergus,Not in a city,61.7104,-142.986,Wrangell-St. Elias National Park & Preserve
36329,47.048537,-109.455315,2019-01-15,28.0,Lewistown,303 East Aztec Drive Lewistown MT 59457,Montana,Fergus,Not in a city,44.4280,-110.588,Yellowstone National Park
36330,47.048537,-109.455315,2019-01-15,28.0,Lewistown,303 East Aztec Drive Lewistown MT 59457,Montana,Fergus,Not in a city,37.8651,-119.538,Yosemite National Park


In [11]:
pollution_df_unclean = pollution_merged_df[
    ((abs(pollution_merged_df["Latitude_x"] - pollution_merged_df["Latitude_y"])) <= 0.44) &\
    ((abs(pollution_merged_df["Longitude_x"] - pollution_merged_df["Longitude_y"])) <= 0.44)
]

In [12]:
pollution_df_unclean

,Latitude_x,Longitude_x,date_local,aqi,local_site_name,site_address,state,county,city,Latitude_y,Longitude_y,Park
248,44.377050,-68.260900,2019-01-15,32.0,MCFARLAND HILL Air Pollutant Research Site,567 Eagle Lake Road,Maine,Hancock,Not in a city,44.3386,-68.273,Acadia National Park
1158,37.198400,-108.490460,2019-01-15,40.0,Mesa Verde NP - Resource Management Area,"MESA VERDE NATIONAL PARK, COLORADO",Colorado,Montezuma,Not in a city,37.2309,-108.462,Mesa Verde National Park
1242,38.458320,-109.821260,2019-01-15,29.0,Canyonlands NP - Island in the Sky,"CANYONLANDS NATIONAL PARK, UTAH",Utah,San Juan,Not in a city,38.7331,-109.593,Arches National Park
1248,38.458320,-109.821260,2019-01-15,29.0,Canyonlands NP - Island in the Sky,"CANYONLANDS NATIONAL PARK, UTAH",Utah,San Juan,Not in a city,38.3269,-109.878,Canyonlands National Park
1716,36.796667,-108.473138,2019-01-15,28.0,1H Substation,"Shiprock Electrical Substation, Waterflow, NM",New Mexico,San Juan,Not in a city,37.2309,-108.462,Mesa Verde National Park
...,...,...,...,...,...,...,...,...,...,...,...,...
32972,32.380820,-111.127160,2019-01-15,19.0,COACHLINE,9597 N. COACHLINE BLVD,Arizona,Pima,Marana,32.2967,-111.167,Saguaro National Park
33034,32.172995,-110.980134,2019-01-15,18.0,ROSE ELEMENTARY,710 W. MICHIGAN ST.,Arizona,Pima,Tucson,32.2967,-111.167,Saguaro National Park
33592,31.879520,-110.996440,2019-01-15,25.0,GREEN VALLEY -REPLACES SITE 0007 245 W ESPERANZA,601 NORTH LA CANADA DRIVE,Arizona,Pima,Green Valley,32.2967,-111.167,Saguaro National Park
34524,38.523100,-78.434710,2019-01-15,37.0,Shenandoah NP - Big Meadows,SHENANDOAH NP BIG MEADOWS,Virginia,Madison,Not in a city,38.2928,-78.680,Shenandoah National Park


In [13]:
pollution_wanted_unclean = pollution_df_unclean.drop(columns = {"local_site_name", "site_address",\
                                                            "state", "county",\
                                                            "city"})
pollution_dates = pd.to_datetime(pollution_wanted_unclean["date_local"])
pollution_date_df_bad = pd.DataFrame(pollution_dates)
pollution_date_df = pollution_date_df_bad.rename(columns = {"date_local": "Year"})
pollution_date_drop = pollution_wanted_unclean.drop(columns= "date_local")
pollution_df_clean = pollution_date_drop.merge(pollution_date_df, left_index = True, right_index = True)

In [14]:
pollution_df_clean

,Latitude_x,Longitude_x,aqi,Latitude_y,Longitude_y,Park,Year
248,44.377050,-68.260900,32.0,44.3386,-68.273,Acadia National Park,2019-01-15
1158,37.198400,-108.490460,40.0,37.2309,-108.462,Mesa Verde National Park,2019-01-15
1242,38.458320,-109.821260,29.0,38.7331,-109.593,Arches National Park,2019-01-15
1248,38.458320,-109.821260,29.0,38.3269,-109.878,Canyonlands National Park,2019-01-15
1716,36.796667,-108.473138,28.0,37.2309,-108.462,Mesa Verde National Park,2019-01-15
...,...,...,...,...,...,...,...
32972,32.380820,-111.127160,19.0,32.2967,-111.167,Saguaro National Park,2019-01-15
33034,32.172995,-110.980134,18.0,32.2967,-111.167,Saguaro National Park,2019-01-15
33592,31.879520,-110.996440,25.0,32.2967,-111.167,Saguaro National Park,2019-01-15
34524,38.523100,-78.434710,37.0,38.2928,-78.680,Shenandoah National Park,2019-01-15


In [15]:
pollution_pivot = pd.pivot_table(pollution_df_clean, values = "aqi", index = ["Year"], columns = ["Park"])

In [16]:
pollution_pivot

Park,Acadia National Park,Arches National Park,Badlands National Park,Big Bend National Park,Biscayne National Park,Black Canyon of the Gunnison National Park,Canyonlands National Park,Carlsbad Caverns National Park,Channel Islands National Park,Congaree National Park,...,Rocky Mountain National Park,Saguaro National Park,Sequoia & Kings Canyon National Parks,Shenandoah National Park,Theodore Roosevelt National Park,Voyageurs National Park,Wind Cave National Park,Yellowstone National Park,Yosemite National Park,Zion National Park
Year,,,,,,,,,,,,,,,,,,,,,
2019-01-15,32.0,29.0,32.0,18.0,35.5,33.0,29.0,23.0,35.0,15.666667,...,44.0,20.5,31.0,37.0,28.0,34.0,37.0,39.0,37.0,26.5
